<h1>Testing the E2E simulations</h1>

## -- JWST aperture --

This script introduces the end-to-end (E2E) simulations that are used in **`calibration.py`**, for the influence calibration of each individual segment. The testing of the script itself is done in this next notebook.

In [ ]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from astropy.io import fits
import astropy.units as u
import webbpsf

os.chdir('../../pastis/')
from config import CONFIG_INI
import util_pastis as util
import image_pastis as impastis

# Path to all the outputs from "aperture_definition.py".
dir = '/Users/ilaginja/Documents/data_from_repos/pastis_data/active/calibration'
# Change into that directory
os.chdir(dir)

os.environ['WEBBPSF_PATH'] = CONFIG_INI.get('local', 'webbpsf_data_path')
print('Currenlty running on WebbPSF', webbpsf.version.version)

In [ ]:
# Get some parameters
which_tel = CONFIG_INI.get('telescope', 'name')
fpm = CONFIG_INI.get(which_tel, 'focal_plane_mask')         # focal plane mask
lyot_stop = CONFIG_INI.get(which_tel, 'pupil_plane_stop')   # Lyot stop
filter = CONFIG_INI.get(which_tel, 'filter_name')                       # filter
im_size_e2e = CONFIG_INI.getint('numerical', 'im_size_px_webbpsf')          # image size in pixels
wss_segs = webbpsf.constants.SEGNAMES_WSS_ORDER
nb_seg = CONFIG_INI.getint(which_tel, 'nb_subapertures')
zern_max = CONFIG_INI.getint('zernikes', 'max_zern')
inner_wa = CONFIG_INI.getint(which_tel, 'IWA')
outer_wa = CONFIG_INI.getint(which_tel, 'OWA')
sampling = CONFIG_INI.getfloat(which_tel, 'sampling')            # sampling

nm_aber = CONFIG_INI.getfloat('calibration', 'single_aberration') * u.nm   # [nm] amplitude of aberration
zern_number = CONFIG_INI.getint('calibration', 'zernike')                  # Which (Noll) Zernike we are calibrating for
wss_zern_nb = util.noll_to_wss(zern_number)                                # Convert from Noll to WSS framework

For starters, lets completely independently create some WebbPSF images of a direct image (no coronagraph) and a coronagraphic image.

In [ ]:
# Create two NIRCam objects
nc = webbpsf.NIRCam()
nc_coro = webbpsf.NIRCam()

In [ ]:
# Btw:
print('NIRCam pixelscale:', nc.pixelscale)
print('Telescope:', nc.telescope)
print('nc name:', nc.name)
print('NIRCam module used:', nc.module)
print('NIRCam list of image masks:', nc.image_mask_list)
print('NIRCam list of pupil masks:', nc.pupil_mask_list)
print('NIRCam currently used OPD:', nc.pupilopd)
print('NIRCam detector list:', nc.detector_list)
print('nc used detector:', nc.detector)
print('Pixel position in (X, Y) on the detector:', nc.detector_position)
print('NIRCam filter list:', nc.filter_list)
print('nc used filter:', nc.filter)
print('nc channel used:', nc.channel)


In [ ]:
# Some displays
plt.figure(figsize=(19, 19))
nc.display()
plt.show()

#nc.calc_psf?
#nc.calcPSF?

In [ ]:
# Show the pupil used
nc_pup = fits.getdata(nc.pupil)
plt.imshow(nc_pup)
plt.title('WebbPSF NIRCam pupil')
plt.show()

print('Pupil shape:', nc_pup.shape)

We can see here how big the pupil array is in terms of pixels that is used in the E2E simulations. The pupil we generate in "aperture_generation.py" for PASTIS needs to have the same pupil array size! Eventually, this will be a number that we enter into the configfile. Currently, the PASTIS image size *im_size_pastis* and the pupil size are the same.

In [ ]:
# Null the OTE OPDs for the PSFs, and also the science instrument (SI) internal WFE.
nc, ote = webbpsf.enable_adjustable_ote(nc)                     # create OTE for default PSF
nc_coro, ote_coro = webbpsf.enable_adjustable_ote(nc_coro)      # create OTE for coronagraph
ote.zero()                       # set OTE for direct PSF to zero
ote_coro.zero()                  # set OTE for coronagraph to zero
nc.include_si_wfe= False         # set SI internal WFE to zero
nc_coro.include_si_wfe= False

In [ ]:
# Display NIRCam isntrument without OTE and SI WFE
plt.figure(figsize=(19, 19))
nc.display()
plt.show()

From the WebbPSF turotial (https://github.com/mperrin/webbpsf/blob/master/notebooks/WebbPSF_tutorial.ipynb) we know that calc_psf() calculates images with different sampling (I think I also explain this in my notebook "DealingWithWebbPSF.ipynb") and we can access them in the different HDU extensions.

In that same notebook, I also explain why I use oversample=1 and nlambda=1 to make the calculations faster.

## NO CORONAGRAPH

### Generating a direct PSF without aberrations

In [ ]:
# Let's see what the current direct PSF looks like (coronagraphic PSF is the same since they've been set up the same
# and I haven't added the coronagraph yet)
psf_direct_hdu = nc.calc_psf(oversample=1, nlambda=1)

In [ ]:
# Display by WebbPSF
plt.figure(figsize=(10,10))
webbpsf.display_psf(psf_direct_hdu)
plt.show()

In [ ]:
# Display with matplotlib
psf_direct = psf_direct_hdu[1].data
print('PSF shape:', psf_direct.shape)
print('PSF max:', np.max(psf_direct))

# Keeping this since I don't tell WebbPSF how big I want my images to be.
# I will start telling it further below though, and then I'll start using
# the zoom() function.
xcen = int(psf_direct.shape[1]/2)
ycen = int(psf_direct.shape[0]/2)
boxhw = 27

plt.figure(figsize=(20,10))
plt.subplot(1, 2, 1)
plt.imshow(psf_direct, norm=LogNorm(), origin='lower')   # WebbPSF uses origin='lower' too, which will
plt.title('Direct PSF')                                  # be important later on with the coronagraphic images
plt.colorbar()
plt.subplot(1, 2, 2)
plt.imshow(psf_direct[ycen-boxhw:ycen+boxhw, xcen-boxhw:xcen+boxhw], norm=LogNorm(), origin='lower')
plt.title('Zoomed in')
plt.show()

We need to make a wavelength and filter choice:

In [ ]:
# Add the filter we want to use
nc.filter = filter
nc_coro.filter = filter

# So far both nc objects are still the same, so I'll display only one
psf = nc.calc_psf(oversample=1, nlambda=1)
plt.figure(figsize=(20,10))
plt.subplot(1, 2, 1)
webbpsf.display_psf(psf)

psf = psf[1].data

# Still using the default image size from WebbPSF
xcen = int(psf.shape[1]/2)
ycen = int(psf.shape[0]/2)
boxhw = 27

plt.subplot(1, 2, 2)
plt.imshow(psf[ycen-boxhw:ycen+boxhw, xcen-boxhw:xcen+boxhw], norm=LogNorm(), origin='lower')
plt.title('Direct PSF')
plt.colorbar()
plt.show()

print('Max of direct PSF:', np.max(psf))

We want our images to be the same size like our simulations, so we use "fov_pixels".

In [ ]:
# Both nc (non-coro and coro) objects are still the same, so I'll display only one.
# Now we're using our custom image size *im_size_e2e*.
psf = nc.calc_psf(fov_pixels=im_size_e2e, oversample=1, nlambda=1)
webbpsf.display_psf(psf)
plt.show()

We want to have normalized images, normalized to the non-coronagraphic, non-aberrated (meaning no sgment is actively moved) PSF that is displayed above. **normp** will be our normalization factor.

In [ ]:
normp = np.max(psf_direct)

Remember what the differend hdu extensions in the WebbPSF images are:
- If oversample = 1: image calculation with detector sampling, and extension 0 andn 1 are the same
- if oversample > 1: image calculation will be done with increased sampling and then binned down to detector sampling. This will make the calculation more accurate, since JWST observations will do things like dithering in order to make images better. Has to be done because some detectors in some wavelengths don't even have Nyquist sampling. Then ext=1 is oversampled image and ext=0 is binned image.

In [ ]:
# Look at the different extensions of the WebbPSF image
psf.info()
webbpsf.display_psf(psf, ext=3)
plt.show()

In [ ]:
# Extract the numpy array
psf = psf[1].data

# Normalize the PSF
psf = psf/normp
print('Done')

In [ ]:
# Display with matplotlibpsf = psf[1].data

# Now starting to use zoom_cen()
boxhw = 27

plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.imshow(psf, norm=LogNorm(), origin='lower')
plt.title('Direct PSF')
plt.colorbar()
plt.subplot(1, 2, 2)
plt.imshow(util.zoom_cen(psf, boxhw), norm=LogNorm(), origin='lower')
plt.title('Direct PSF - zoomed in')
plt.show()

print('Total image shape:', psf.shape)
print('PSF max:', np.max(psf))

### A single aberrated segment

To compare to the analytical images step by step, I will first create images with only one segment aberrated.

In [ ]:
segnum = 5     # Which segment are we aberrating - I number them starting with 1
segnum -= 1    # Which is why I have to subtract one, because WebbPSF starts numbering them at 0
#nm_aber = 100 # in in put units
# Extract the correct segment name from WebbPSF
seg = wss_segs[segnum].split('-')[0]
print('Aberrated segment:', seg)

In [ ]:
# Create arrays to hold Zernike aberration coefficients
Aber_WSS = np.zeros([nb_seg, zern_max])           # The Zernikes here will be filled in the WSS order!!!
                                                  # Because it goes into _apply_hexikes_to_seg().

# Feed the aberration nm_aber into the array position
# that corresponds to the correct Zernike, but only on segment i
Aber_WSS[segnum, wss_zern_nb-1] = nm_aber.to(u.m).value   # Aberration on the segment we're currently working on;
                                                          # convert to meters; -1 on the Zernike because Python starts
                                                          # numbering at 0.

#-# Crate OPD with aberrated segment, NO CORONAGRAPH
print('Applying aberration to OTE.')
print('nm_aber: {}'.format(nm_aber))
ote.reset()   # Making sure there are no previous movements on the segments.
ote.zero()    # For now, ignore internal WFE.
ote._apply_hexikes_to_seg(seg, Aber_WSS[segnum,:])

In [ ]:
# Display the OTE
ote.display_opd()
plt.show()
# At this point, WebbPSF still numbers the segments wrong in the exit pupil,
# so it's the easiest to orient yourself by the spiders.

In [ ]:
# Calculate the PSF
psf_minizern = nc.calc_psf(fov_pixels=im_size_e2e, oversample=1, nlambda=1)
webbpsf.display_psf(psf_minizern)
plt.show()

psf_minizern = psf_minizern[1].data/normp

In [ ]:
# Display with matplotlib
boxhw = 27
print('psf_minizern.shape:', psf_minizern.shape)

plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.imshow(psf_minizern, norm=LogNorm(), origin='lower')
plt.title('Equivalent of Envelope from mini Zernike')
plt.colorbar()
plt.subplot(1, 2, 2)
plt.imshow(util.zoom_cen(psf_minizern, boxhw), norm=LogNorm(), origin='lower')
plt.title('Envelope - zoomed in')
plt.show()

Compare this image with one single aberrated segment vs. the non-aberrated PSF:

In [ ]:
# Subtract the pserfect direct PSF off the single-segment aberrated PSF
one_aber_residual = psf - psf_minizern

plt.figure(figsize=(20, 10))
plt.subplot(1, 3, 1)
plt.imshow(util.zoom_cen(psf, boxhw), norm=LogNorm(), origin='lower')
plt.title('Direct PSF, perfect')
#plt.colorbar()
plt.subplot(1, 3, 2)
plt.imshow(util.zoom_cen(psf_minizern, boxhw), norm=LogNorm(), origin='lower')
plt.title('Direct PSF one aberrated segment')
#plt.colorbar()
plt.subplot(1, 3, 3)
plt.imshow(util.zoom_cen(one_aber_residual, boxhw), norm=LogNorm(), origin='lower')
plt.title('Residual')
#plt.colorbar()
plt.show()

In [ ]:
# Repeat on a smaller image direclty instead of cropping it afterwards, for faster computation
# Calculate the PSF
psf_minizern = nc.calc_psf(fov_pixels=54, oversample=1, nlambda=1)
webbpsf.display_psf(psf_minizern)
plt.show()

psf_minizern = psf_minizern[1].data/normp

# Display with matplotlib
boxhw = 27

plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.imshow(psf_minizern, norm=LogNorm(), origin='lower')
plt.title('PSF')
plt.colorbar()
plt.show()

In [ ]:
"""
# Make a loop over the first eight Zernike envelopes, like in notebook 2
aber_wss_loop = np.zeros([nb_seg, 8])
psfs_env = []
plt.figure(figsize=(18, 60))
plt.suptitle('Different Zernikes envelopes from WebbPSF')
noll_as_wss = np.array([1, 3, 2, 5, 4, 6, 7, 8]) #, 11, 9, 10])    # reordering Noll Zernikes to WSS, for ease of use
print('nm_aber:', nm_aber, 'in input units')

for i, zern in enumerate(noll_as_wss):
    
    # Put the Zernike coefficient in correct place in
    aber_wss_loop[:,:] = 0     # set all entries to zero
    aber_wss_loop[segnum, zern-1] = nm_aber / aber_u   # fill only the index for current Zernike, in meters
    #print(aber_wss_loop[segnum, :])
    
    # Put Zernike on correct segment on OTE
    ote.reset()   # Making sure there are no previous movements on the segments.
    ote.zero()    # For now, ignore internal WFE.
    ote._apply_hexikes_to_seg(seg, aber_wss_loop[segnum,:])
    
    # Display the OTE
    plt.subplot(8, 2, i*2+1)
    ote.display_opd()
    
    # Calculate the PSF
    print('Calculating PSF', str(i+1) + '/' + '8')
    psf_zernloop = nc.calc_psf(fov_pixels=54, oversample=1, nlambda=1)
    psf_zernloop = psf_zernloop[1].data
    psfs_env.append(psf_zernloop)
    
    # Display the PSF
    plt.subplot(8, 2, i*2+2)
    plt.imshow(psf_zernloop, norm=LogNorm(), origin='lower')
    
plt.show()
psfs_env = np.array(psfs_env)
"""

# This was a thinking mistake from my side. I tried modeling the single Zernike envelope
# from the analytical model, which I can't do direcly in this simulation because I only
# have access to the full aperture.

# But I'll keep the code, because you never know what it could be useful for.

In [ ]:
"""
# Display them
plt.figure(figsize=(16, 8))
for i in range(noll_as_wss.shape[0]):
    plt.subplot(2, 4, i+1)
    plt.imshow(psfs_env[i], norm=LogNorm(), origin='lower')
    plt.title('Noll Zernike: ' + str(i+1))
plt.show()
"""

### Pair-wise aberrated segments

In [ ]:
# Decide which two segments you want to aberrate
segnum1 = 8     # Which segments are we aberrating - I number them starting with 1
segnum2 = 16

segnum_array = np.array([segnum1, segnum2])
segnum_array -= 1    # Which is why I have to subtract one, because WebbPSF starts numbering them at 0

zern_pair = 1  # Which Noll Zernike are we putting on the segments.

# Extract the correct segment names from WebbPSF
seg_array = []
for i, senu in enumerate(segnum_array):
    seg_array.append(wss_segs[senu].split('-')[0])

seg_array = np.array(seg_array)
print('Aberration: {}'.format(nm_aber))
print('Aberrated segments:', seg_array)
print('Noll Zernike used:', zern_pair)

In [ ]:
aber_wss_loop = np.zeros([nb_seg, 8])
noll_as_wss = np.array([1, 3, 2, 5, 4, 6, 7, 8]) #, 11, 9, 10])    # reordering Noll Zernikes to WSS, for ease of use
print('nm_aber: {}'.format(nm_aber))

# Apply aberration to all sgements
ote.reset()   # Making sure there are no previous movements on the segments.
ote.zero()    # For now, ignore internal WFE.
for i, nseg in enumerate(seg_array):
    aber_wss_loop[segnum_array[i], noll_as_wss[zern_pair-1]-1] = nm_aber.to(u.m).value   # fill only the index for current Zernike, in meters

    # Put Zernike on correct segments on OTE
    ote._apply_hexikes_to_seg(nseg, aber_wss_loop[segnum_array[i],:])

# Display the OTE
ote.display_opd()
plt.show()

In [ ]:
# Calculate the PSF
psf_zernpair= nc.calc_psf(fov_pixels=154, oversample=1, nlambda=1)   # oversampled for beeter seeign the fringes
psf_zernpair = psf_zernpair[0].data/normp                            # getting the oversampled extension
    
# Display the PSF
plt.figure(figsize=(10, 10))
plt.subplot(1, 1, 1)
plt.imshow(psf_zernpair, norm=LogNorm(), origin='lower')
plt.title('Direct PSF of a pair-wise aberrated segmented OTE')
plt.colorbar()
plt.show()

print(psf_zernpair.shape)

I'm gonna stop here and go back to do the same thing with the analytical model in notebook 2. I am not sure the effect in the focal plane of me aberrating a pair of segments is really what it's supposed to be.

I created some images from specific pairs and then saved them:

In [ ]:
#segs_3_11_noll_1_dir = np.copy(psf_zernpair)
#segs_11_17_noll_1_dir = np.copy(psf_zernpair)
#segs_6_11_noll_1_dir = np.copy(psf_zernpair)
#segs_9_2_noll_1_dir = np.copy(psf_zernpair)
#segs_9_5_noll_1_dir = np.copy(psf_zernpair)
#segs_9_15_noll_1_dir = np.copy(psf_zernpair)
#segs_8_1_noll_1_dir = np.copy(psf_zernpair)
#segs_8_6_noll_1_dir = np.copy(psf_zernpair)
#segs_8_16_noll_1_dir = np.copy(psf_zernpair)

In [ ]:
save_dir1 = '/astro/opticslab1/PASTIS/jwst_data/E2E_pair_aberrations/2019-1-25-16h-18min_piston_100nm'
# util.write_fits(segs_3_11_noll_1_dir, os.path.join(save_dir1, 'segs_3_11_noll_1_dir.fits'))
# util.write_fits(segs_11_17_noll_1_dir, os.path.join(save_dir1, 'segs_11_17_noll_1_dir.fits'))
# util.write_fits(segs_6_11_noll_1_dir, os.path.join(save_dir1, 'segs_6_11_noll_1_dir.fits'))
# util.write_fits(segs_9_2_noll_1_dir, os.path.join(save_dir1, 'segs_9_2_noll_1_dir.fits'))
# util.write_fits(segs_9_5_noll_1_dir, os.path.join(save_dir1, 'segs_9_5_noll_1_dir.fits'))
# util.write_fits(segs_9_15_noll_1_dir, os.path.join(save_dir1, 'segs_9_15_noll_1_dir.fits'))
# util.write_fits(segs_8_1_noll_1_dir, os.path.join(save_dir1, 'segs_8_1_noll_1_dir.fits'))
# util.write_fits(segs_8_6_noll_1_dir, os.path.join(save_dir1, 'segs_8_6_noll_1_dir.fits'))
# util.write_fits(segs_8_16_noll_1_dir, os.path.join(save_dir1, 'segs_8_16_noll_1_dir.fits'))

In general, I will have to load these images from central store:
- '/astro/opticslab1/PASTIS/jwst_data/E2E_pair_aberrations/2019-1-18-17h-5min_piston_1000nm_pairs' will have images generated with aberrations of 1000 nm per segment which is too much compared to JWST's wavelength and the sort of aberrations that are expected in-flight
- '/astro/opticslab1/PASTIS/jwst_data/E2E_pair_aberrations/2019-1-25-16h-18min_piston_100nm' has images generated with aberrations of 100 nm per segment, but this aberration is not high enough to make us see the fringes


In [ ]:
read_dir1 = '/astro/opticslab1/PASTIS/jwst_data/E2E_pair_aberrations/2019-1-18-17h-5min_piston_1000nm_pairs'
segs_3_11_noll_1_dir = fits.getdata(os.path.join(read_dir1, 'segs_3_11_noll_1_dir.fits'))
segs_11_17_noll_1_dir = fits.getdata(os.path.join(read_dir1, 'segs_11_17_noll_1_dir.fits'))
segs_6_11_noll_1_dir = fits.getdata(os.path.join(read_dir1, 'segs_6_11_noll_1_dir.fits'))
segs_9_2_noll_1_dir = fits.getdata(os.path.join(read_dir1, 'segs_9_2_noll_1_dir.fits'))
segs_9_5_noll_1_dir = fits.getdata(os.path.join(read_dir1, 'segs_9_5_noll_1_dir.fits'))
segs_9_15_noll_1_dir = fits.getdata(os.path.join(read_dir1, 'segs_9_15_noll_1_dir.fits'))
segs_8_1_noll_1_dir = fits.getdata(os.path.join(read_dir1, 'segs_8_1_noll_1_dir.fits'))
segs_8_6_noll_1_dir = fits.getdata(os.path.join(read_dir1, 'segs_8_6_noll_1_dir.fits'))
segs_8_16_noll_1_dir = fits.getdata(os.path.join(read_dir1, 'segs_8_16_noll_1_dir.fits'))

Let's have a look at some of the images (refer to numbered pupil to identify baslines these correspond to).

In [ ]:
# Gotta check how big the loaded images are!
print('Loaded images shape:', segs_3_11_noll_1_dir.shape)
print('im_size_e2e:', im_size_e2e)

# If im_size_e2e is bigger than images we loaded, this won't work
# and you have to define a box half-size manually for imwidth.
boxw = int(im_size_e2e/2)
boxw2 = boxw/2
    
if im_size_e2e < segs_3_11_noll_1_dir.shape[0]:
    imwidth = bozw2
else:
    #raise Exception('! You have to set imwidth manually ! And then comment this line out.')
    pass

# Chose what image size (in pixels) we want to display
imwidth = 40

plt.figure(figsize=(18, 12))
plt.suptitle('Pair-wise aberrations on direct (no coro) WebbPSF images')
plt.subplot(2, 3, 1)
plt.imshow(util.zoom_cen(segs_3_11_noll_1_dir, imwidth), norm=LogNorm(), origin='lower')
plt.title('Piston on segments 3 and 11')

plt.subplot(2, 3, 2)
plt.imshow(util.zoom_cen(segs_6_11_noll_1_dir, imwidth), norm=LogNorm(), origin='lower')
plt.title('Piston on segments 6 and 11')

plt.subplot(2, 3, 3)
plt.imshow(util.zoom_cen(segs_11_17_noll_1_dir, imwidth), norm=LogNorm(), origin='lower')
plt.title('Piston on segments 11 and 17')

plt.subplot(2, 3, 4)
plt.imshow(util.zoom_cen(segs_9_2_noll_1_dir, imwidth), norm=LogNorm(), origin='lower')
plt.title('Piston on segments 9 and 2')

plt.subplot(2, 3, 5)
plt.imshow(util.zoom_cen(segs_9_5_noll_1_dir, imwidth), norm=LogNorm(), origin='lower')
plt.title('Piston on segments 5 and 9')

plt.subplot(2, 3, 6)
plt.imshow(util.zoom_cen(segs_9_15_noll_1_dir, imwidth), norm=LogNorm(), origin='lower')
plt.title('Piston on segments 9 and 15')

plt.show()

I made sure to make images from the same aberrated pairs like in the analytical notebook (notebook 2), so we can compare them here now.

In [ ]:
# Load the analytical images
read_dir_ana = '/astro/opticslab1/PASTIS/jwst_data/uncalibrated_analytical_images/2018-01-19-18h-31min_piston_1000nm_exitpupil'
segs_3_11_noll_1_ana = fits.getdata(os.path.join(read_dir_ana, 'segs_3_11_noll_1.fits'))
segs_11_17_noll_1_ana = fits.getdata(os.path.join(read_dir_ana, 'segs_11_17_noll_1.fits'))
segs_6_11_noll_1_ana = fits.getdata(os.path.join(read_dir_ana, 'segs_6_11_noll_1.fits'))
segs_9_2_noll_1_ana = fits.getdata(os.path.join(read_dir_ana, 'segs_9_2_noll_1.fits'))
segs_9_5_noll_1_ana = fits.getdata(os.path.join(read_dir_ana, 'segs_9_5_noll_1.fits'))
segs_9_15_noll_1_ana = fits.getdata(os.path.join(read_dir_ana, 'segs_9_15_noll_1.fits'))
segs_8_1_noll_1_ana = fits.getdata(os.path.join(read_dir_ana, 'segs_8_1_noll_1.fits'))
segs_8_6_noll_1_ana = fits.getdata(os.path.join(read_dir_ana, 'segs_8_6_noll_1.fits'))
segs_8_16_noll_1_ana = fits.getdata(os.path.join(read_dir_ana, 'segs_8_16_noll_1.fits'))

Compare pairs **3-11**, **6-11** and **11-17** between E2E and analytical:

In [ ]:
# Chose what image size (in pixels) we want to display
imwidth = imwidth

plt.figure(figsize=(18, 12))
plt.suptitle('Comparison of E2E and analtical DIRECT images')
plt.subplot(2, 3, 1)
plt.imshow(util.zoom_cen(segs_3_11_noll_1_dir, imwidth), norm=LogNorm(), origin='lower')
plt.title('Piston on segments 3 and 11 - E2E')

plt.subplot(2, 3, 2)
plt.imshow(util.zoom_cen(segs_6_11_noll_1_dir, imwidth), norm=LogNorm(), origin='lower')
plt.title('Piston on segments 6 and 11 - E2E')

plt.subplot(2, 3, 3)
plt.imshow(util.zoom_cen(segs_11_17_noll_1_dir, imwidth), norm=LogNorm(), origin='lower')
plt.title('Piston on segments 11 and 17 - E2E')

plt.subplot(2, 3, 4)
plt.imshow(util.zoom_cen(segs_3_11_noll_1_ana, imwidth), norm=LogNorm(), origin='lower')
plt.title('Piston on segments 3 and 11 - analytical')

plt.subplot(2, 3, 5)
plt.imshow(util.zoom_cen(segs_6_11_noll_1_ana, imwidth), norm=LogNorm(), origin='lower')
plt.title('Piston on segments 5 and 11 - analytical')

plt.subplot(2, 3, 6)
plt.imshow(util.zoom_cen(segs_11_17_noll_1_ana, imwidth), norm=LogNorm(), origin='lower')
plt.title('Piston on segments 11 and 17 - analytical')

plt.show()

There is clearly a lot more going on in the WebbPSF images, since they will have incorporated many additional effects compared to the analytical images. But, foro the same aberrated segment, we can see fringes of the same structure and orientation, so I think this is fine!

## WITH CORONAGRAPH

### Generating a coronagraphic PSF without aberrations

In [ ]:
# Now add the coronagraph to nc_coro
nc_coro.image_mask = fpm
nc_coro.pupil_mask = lyot_stop

# And show what that looks like
plt.figure(figsize=(18, 9))
psf_coro = nc_coro.calc_psf(fov_pixels=im_size_e2e, oversample=1, nlambda=1, display=True)
plt.show()
psf_coro_im = psf_coro[1].data/normp

print('PSF calculation done')

In [ ]:
# I can't use webbpsf.display_psf(psf_coro) because I couldn't figure out how to change the color scaling
# and it turns out all black. So I'll just use matplotlib.
plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.imshow(psf_coro_im, norm=LogNorm(), origin='lower')
plt.title('Coronagraphic PSF - zoomed in')
plt.subplot(1, 2, 2)
plt.imshow(util.zoom_cen(psf_coro_im, 70), norm=LogNorm(), origin='lower')
plt.title('Coronagraphic PSF')
plt.colorbar()
plt.show()

Let's confirm what image size we're using. The NIRCam field of view is 20'' and the plate scale in the long wavelength channel is 0.063''/pixel. This means if we divide 20 by 0.063, we can tell how big the total FoV is in NIRCam pixels.

In [ ]:
print('NIRCam images will have', 20/0.063, 'pixels on either side of the detector.')

And we just rounded up to 320 pixels. For comparison here, we'll show the WebbPSF native display that gives you the image in terms of arcseconds - and you should see a 20'' x 20'' field of view (going from -10'' to 10'' on both axes).

In [ ]:
# For comparison, the webbpsf display in physical units for the fov:
# Also, I have figured out here how to change the image scale.
plt.figure(figsize=(10, 10))
webbpsf.display_psf(psf_coro, vmin=1e-12, vmax=1e-6)
plt.show()

This is the place where we can see that in order to match our matplotlib displays of the PSF with that of WebbPSF, we need to use the keyword "origin='lower'" in imshow().

### A single aberrated segment

For the calibration of the analytical images, I need to create images that stem for the pupil having one single aberrated segment.

In [ ]:
# Define what segment to aberrate
segnum = 5     # Which segment are we aberrating - I number them starting with 1
segnum -= 1    # Which is why I have to subtract one, because WebbPSF starts numbering them at 0
# Extract the correct segment name from WebbPSF
seg = wss_segs[segnum].split('-')[0]
print('Aberrated segment:', seg)

# Define what Noll Zernike we're using
zern_number = 1
wss_zern_nb = util.noll_to_wss(zern_number)  

# Maybe play around with amount of aberration
#nm_aber = 1000.     # in input units

# Create arrays to hold Zernike aberration coefficients
Aber_WSS = np.zeros([nb_seg, zern_max])           # The Zernikes here will be filled in the WSS order!!!
                                                  # Because it goes into _apply_hexikes_to_seg().

# Feed the aberration nm_aber into the array position
# that corresponds to the correct Zernike, but only on segment i
Aber_WSS[segnum, wss_zern_nb-1] = nm_aber.to(u.m).value     # Aberration on the segment we're currently working on;
                                                            # convert to meters; -1 on the Zernike because Python starts
                                                            # numbering at 0.

#-# Crate OPD with aberrated segment, NO CORONAGRAPH
print('Applying aberration to OTE.')
print('nm_aber: {}'.format(nm_aber))
ote_coro.reset()   # Making sure there are no previous movements on the segments.
ote_coro.zero()    # For now, ignore internal WFE.
ote_coro._apply_hexikes_to_seg(seg, Aber_WSS[segnum,:])

# Display the OTE
ote_coro.display_opd()
plt.show()
# At this point, WebbPSF still numbers the segments wrong in the exit pupil,
# so it's the easiest to orient yourself by the spiders.

In [ ]:
# Calculate the PSF
psf_single_coro = nc_coro.calc_psf(fov_pixels=im_size_e2e, oversample=1, nlambda=1)
plt.figure(figsize=(10, 10))
webbpsf.display_psf(psf_single_coro, vmin=1e-12, vmax=1e-6)
plt.show()

psf_single_coro = psf_single_coro[1].data/normp

In [ ]:
# Display with matplotlib
boxhw = im_size_e2e/2
box2 = boxhw/2
print('nm_aber: {}'.format(nm_aber))

plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.imshow(psf_single_coro, norm=LogNorm(), origin='lower')
plt.title('One aberrated segment in coronagraphic setup')
plt.colorbar()
plt.subplot(1, 2, 2)
plt.imshow(util.zoom_cen(psf_single_coro, box2), norm=LogNorm(), origin='lower')
plt.title('Zoomed in')
plt.show()

For piston, an aberration of 10 nm shows no effect, 100 nm already visibly messes the PSF up and 1000 nm make a very distinct change to the PSF, probably too much for PASTIS purposes.

### Pair-wise aberrated segments with coronagraph

In [ ]:
# Decide which two segments you want to aberrate
segnum1 = 8     # Which segments are we aberrating - I number them starting with 1
segnum2 = 16

# Segment aberrations are additive, so if you use a segment number twice, the
# aberration will be applied twice!

segnum_array = np.array([segnum1, segnum2])
segnum_array -= 1    # Which is why I have to subtract one, because WebbPSF starts numbering them at 0

zern_pair = 1  # Which Noll Zernike are we putting on the segments.

# Extract the correct segment names from WebbPSF
seg_array = []
for i, senu in enumerate(segnum_array):
    seg_array.append(wss_segs[senu].split('-')[0])

seg_array = np.array(seg_array)
print('Aberration used: {}'.format(nm_aber))
print('Aberrated segments:', seg_array)
print('Noll Zernike used:', zern_pair)

In [ ]:
aber_wss_loop = np.zeros([nb_seg, 8])
noll_as_wss = np.array([1, 3, 2, 5, 4, 6, 7, 8]) #, 11, 9, 10])    # reordering Noll Zernikes to WSS, for ease of use
print('nm_aber: {}'.format(nm_aber))

# Apply aberration to all sgements
ote_coro.reset()   # Making sure there are no previous movements on the segments.
ote_coro.zero()    # For now, ignore internal WFE.
for i, nseg in enumerate(seg_array):
    aber_wss_loop[segnum_array[i], noll_as_wss[zern_pair-1]-1] = nm_aber.to(u.m).value   # fill only the index for current Zernike, in meters

    # Put Zernike on correct segments on OTE
    ote_coro._apply_hexikes_to_seg(nseg, aber_wss_loop[segnum_array[i],:])

# Display the OTE
ote_coro.display_opd()
plt.show()

In [ ]:
# Calculate the PSF
psf_coro_pair= nc_coro.calc_psf(fov_pixels=im_size_e2e, oversample=1, nlambda=1)
psf_coro_pair = psf_coro_pair[0].data/normp                 # getting the oversampled extension
"""   
# Display the PSF
plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.imshow(psf_coro_pair, norm=LogNorm(), origin='lower')
plt.title('Pair-wise aberrated coronagraphic PSF')
plt.subplot(1, 2, 2)
plt.imshow(util.zoom_cen(psf_coro_pair, box2), norm=LogNorm(), origin='lower')
plt.title('Zoomed')
plt.show()
"""
print('nm_aber: {}'.format(nm_aber))
print('Aberrated segments:', seg_array)
print('Noll Zernike used:', zern_pair)
print(psf_coro_pair.shape)

In [ ]:
# Create DH
dh_area = util.create_dark_hole(psf_coro_pair, inner_wa, outer_wa, sampling)

testim = psf_coro_pair * dh_area

#
contrast = np.mean(testim[np.where(testim != 0)])
print(contrast)

plt.imshow(testim)
plt.show()

In [ ]:
#segs_3_11_noll_1_coro = np.copy(psf_coro_pair)
#segs_11_17_noll_1_coro = np.copy(psf_coro_pair)
#segs_6_11_noll_1_coro = np.copy(psf_coro_pair)
#segs_9_2_noll_1_coro = np.copy(psf_coro_pair)
#segs_9_5_noll_1_coro = np.copy(psf_coro_pair)
#segs_9_15_noll_1_coro = np.copy(psf_coro_pair)
#segs_8_1_noll_1_coro = np.copy(psf_coro_pair)
#segs_8_6_noll_1_coro = np.copy(psf_coro_pair)
#segs_8_16_noll_1_coro = np.copy(psf_coro_pair)

In [ ]:
# Save to central store
save_dir1 = '/astro/opticslab1/PASTIS/jwst_data/E2E_pair_aberrations/2019-1-22-9h-53min'
#util.write_fits(segs_3_11_noll_1_coro, os.path.join(save_dir1, 'segs_3_11_noll_1_coro.fits'))
#util.write_fits(segs_11_17_noll_1_coro, os.path.join(save_dir1, 'segs_11_17_noll_1_coro.fits'))
#util.write_fits(segs_6_11_noll_1_coro, os.path.join(save_dir1, 'segs_6_11_noll_1_coro.fits'))
#util.write_fits(segs_9_2_noll_1_coro, os.path.join(save_dir1, 'segs_9_2_noll_1_coro.fits'))
#util.write_fits(segs_9_5_noll_1_coro, os.path.join(save_dir1, 'segs_9_5_noll_1_coro.fits'))
#util.write_fits(segs_9_15_noll_1_coro, os.path.join(save_dir1, 'segs_9_15_noll_1_coro.fits'))

#util.write_fits(segs_8_1_noll_1_coro, os.path.join(save_dir1, 'segs_8_1_noll_1_coro.fits'))
#util.write_fits(segs_8_6_noll_1_coro, os.path.join(save_dir1, 'segs_8_6_noll_1_coro.fits'))
#util.write_fits(segs_8_16_noll_1_coro, os.path.join(save_dir1, 'segs_8_16_noll_1_coro.fits'))

In [ ]:
# Read from central store

# 1000 nm aberrations:
read_dir1 = '/astro/opticslab1/PASTIS/jwst_data/E2E_pair_aberrations/2019-1-22-9h-53min_coro_piston_1000nm_pairs'

# 100 nm aberrations
#read_dir1 = '/astro/opticslab1/PASTIS/jwst_data/E2E_pair_aberrations/2019-1-25-16h-18min_piston_100nm'

segs_3_11_noll_1_coro = fits.getdata(os.path.join(read_dir1, 'segs_3_11_noll_1.fits'))
segs_11_17_noll_1_coro = fits.getdata(os.path.join(read_dir1, 'segs_11_17_noll_1.fits'))
segs_6_11_noll_1_coro = fits.getdata(os.path.join(read_dir1, 'segs_6_11_noll_1.fits'))
segs_9_2_noll_1_coro = fits.getdata(os.path.join(read_dir1, 'segs_9_2_noll_1.fits'))
segs_9_5_noll_1_coro = fits.getdata(os.path.join(read_dir1, 'segs_9_5_noll_1.fits'))
segs_9_15_noll_1_coro = fits.getdata(os.path.join(read_dir1, 'segs_9_15_noll_1.fits'))

segs_8_1_noll_1_coro = fits.getdata(os.path.join(read_dir1, 'segs_8_1_noll_1.fits'))
segs_8_6_noll_1_coro = fits.getdata(os.path.join(read_dir1, 'segs_8_6_noll_1.fits'))
segs_8_16_noll_1_coro = fits.getdata(os.path.join(read_dir1, 'segs_8_16_noll_1.fits'))

In [ ]:
# Have a look at the images

# Gotta check how big the loaded images are!
print('Loaded images shape:', segs_3_11_noll_1_coro.shape)
print('im_size_e2e:', im_size_e2e)

# If im_size_e2e is bigger than images we loaded, this won't work
# and you have to define a box half-size manually for imwidth.
boxw = int(im_size_e2e/2)
boxw2 = boxw/2
    
if im_size_e2e < segs_3_11_noll_1_dir.shape[0]:
    imwidth = bozw2
else:
    #raise Exception('! You have to set imwidth manually ! And then comment this line out.')
    pass

# Chose what image size (in pixels) we want to display
imwidth = 50

plt.figure(figsize=(18, 12))
plt.suptitle('Pair-wise aberration in coronagraphpic WebbPSF images')
plt.subplot(2, 3, 1)
plt.imshow(util.zoom_cen(segs_3_11_noll_1_coro, imwidth), norm=LogNorm(), origin='lower')
plt.title('Piston on segments 3 and 11')

plt.subplot(2, 3, 2)
plt.imshow(util.zoom_cen(segs_6_11_noll_1_coro, imwidth), norm=LogNorm(), origin='lower')
plt.title('Piston on segments 6 and 11')

plt.subplot(2, 3, 3)
plt.imshow(util.zoom_cen(segs_11_17_noll_1_coro, imwidth), norm=LogNorm(), origin='lower')
plt.title('Piston on segments 11 and 17')

plt.subplot(2, 3, 4)
plt.imshow(util.zoom_cen(segs_9_2_noll_1_coro, imwidth), norm=LogNorm(), origin='lower')
plt.title('Piston on segments 9 and 2')

plt.subplot(2, 3, 5)
plt.imshow(util.zoom_cen(segs_9_5_noll_1_coro, imwidth), norm=LogNorm(), origin='lower')
plt.title('Piston on segments 5 and 9')

plt.subplot(2, 3, 6)
plt.imshow(util.zoom_cen(segs_9_15_noll_1_coro, imwidth), norm=LogNorm(), origin='lower')
plt.title('Piston on segments 9 and 15')

plt.show()

We're missing one intermediate baseline with these combinations though, because we have to skip the center segment. I want to know what that looks like though, so here's some more images.

In [ ]:
# Chose what image size (in pixels) we want to display
imwidth = imwidth

plt.figure(figsize=(18, 6))
plt.suptitle('Pair-wise aberrated coronagraphpic WebbPSF images')
plt.subplot(1, 3, 1)
plt.imshow(util.zoom_cen(segs_8_1_noll_1_coro, imwidth), norm=LogNorm(), origin='lower')
plt.title('Piston on segments 8 and 1')

plt.subplot(1, 3, 2)
plt.imshow(util.zoom_cen(segs_8_6_noll_1_coro, imwidth), norm=LogNorm(), origin='lower')
plt.title('Piston on segments 8 and 6')

plt.subplot(1, 3, 3)
plt.imshow(util.zoom_cen(segs_8_16_noll_1_coro, imwidth), norm=LogNorm(), origin='lower')
plt.title('Piston on segments 8 and 16')

plt.show()

## COMPARING ANALYTICAL, E2E DIRECT AND E2E CORONAGRAPHIC

Display comparison for **piston** with the pairs **9-2**, **9-5** and **9-15**.

In [ ]:
# Chose what image size (in pixels) we want to display
imwidth = imwidth

plt.figure(figsize=(18, 18))
plt.suptitle('Comparison of pair-wise aberration')
plt.subplot(3, 3, 1)
plt.imshow(util.zoom_cen(segs_9_2_noll_1_ana, imwidth), norm=LogNorm(), origin='lower')
plt.title('Analytical - Piston on segments 9 and 2')

plt.subplot(3, 3, 2)
plt.imshow(util.zoom_cen(segs_9_5_noll_1_ana, imwidth), norm=LogNorm(), origin='lower')
plt.title('Analytical - Piston on segments 5 and 9')

plt.subplot(3, 3, 3)
plt.imshow(util.zoom_cen(segs_9_15_noll_1_ana, imwidth), norm=LogNorm(), origin='lower')
plt.title('Analytical - Piston on segments 9 and 15')

plt.subplot(3, 3, 4)
plt.imshow(util.zoom_cen(segs_9_2_noll_1_dir, imwidth), norm=LogNorm(), origin='lower')
plt.title('Direct WebbPSF - Piston on segments 9 and 2')

plt.subplot(3, 3, 5)
plt.imshow(util.zoom_cen(segs_9_5_noll_1_dir, imwidth), norm=LogNorm(), origin='lower')
plt.title('Direct WebbPSF - Piston on segments 5 and 9')

plt.subplot(3, 3, 6)
plt.imshow(util.zoom_cen(segs_9_15_noll_1_dir, imwidth), norm=LogNorm(), origin='lower')
plt.title('Direct WebbPSF - Piston on segments 9 and 15')

plt.subplot(3, 3, 7)
plt.imshow(util.zoom_cen(segs_9_2_noll_1_coro, imwidth), norm=LogNorm(), origin='lower')
plt.title('Coro WebbPSF - Piston on segments 9 and 2')

plt.subplot(3, 3, 8)
plt.imshow(util.zoom_cen(segs_9_5_noll_1_coro, imwidth), norm=LogNorm(), origin='lower')
plt.title('Coro WebbPSF - Piston on segments 5 and 9')

plt.subplot(3, 3, 9)
plt.imshow(util.zoom_cen(segs_9_15_noll_1_coro, imwidth), norm=LogNorm(), origin='lower')
plt.title('Coro WebbPSF - Piston on segments 9 and 15')

plt.show()

Display the comparison for **piston** with the pairs **8-1**, **8-6** and **8-16**.

In [ ]:
# Chose what image size (in pixels) we want to display
imwidth = imwidth

plt.figure(figsize=(18, 18))
plt.suptitle('Comparison of pair-wise aberration')
plt.subplot(3, 3, 1)
plt.imshow(util.zoom_cen(segs_8_1_noll_1_ana, imwidth), norm=LogNorm(), origin='lower')
plt.title('Analytical - Piston on segments 8 and 1')

plt.subplot(3, 3, 2)
plt.imshow(util.zoom_cen(segs_8_6_noll_1_ana, imwidth), norm=LogNorm(), origin='lower')
plt.title('Analytical - Piston on segments 8 and 6')

plt.subplot(3, 3, 3)
plt.imshow(util.zoom_cen(segs_8_16_noll_1_ana, imwidth), norm=LogNorm(), origin='lower')
plt.title('Analytical - Piston on segments 8 and 16')

plt.subplot(3, 3, 4)
plt.imshow(util.zoom_cen(segs_8_1_noll_1_dir, imwidth), norm=LogNorm(), origin='lower')
plt.title('Direct WebbPSF - Piston on segments 8 and 1')

plt.subplot(3, 3, 5)
plt.imshow(util.zoom_cen(segs_8_6_noll_1_dir, imwidth), norm=LogNorm(), origin='lower')
plt.title('Direct WebbPSF - Piston on segments 8 and 6')

plt.subplot(3, 3, 6)
plt.imshow(util.zoom_cen(segs_8_16_noll_1_dir, imwidth), norm=LogNorm(), origin='lower')
plt.title('Direct WebbPSF - Piston on segments 6 and 16')

plt.subplot(3, 3, 7)
plt.imshow(util.zoom_cen(segs_8_1_noll_1_coro, imwidth), norm=LogNorm(), origin='lower')
plt.title('Coro WebbPSF - Piston on segments 8 and 1')

plt.subplot(3, 3, 8)
plt.imshow(util.zoom_cen(segs_8_6_noll_1_coro, imwidth), norm=LogNorm(), origin='lower')
plt.title('Coro WebbPSF - Piston on segments 8 and 69')

plt.subplot(3, 3, 9)
plt.imshow(util.zoom_cen(segs_8_16_noll_1_coro, imwidth), norm=LogNorm(), origin='lower')
plt.title('Coro WebbPSF - Piston on segments 8 and 18')

plt.show()